In [1]:
from agential.cog.critic.strategies.code import CriticCodeStrategy, CritHEvalCodeStrategy
from agential.llm.llm import LLM
from dotenv import load_dotenv
load_dotenv()
import os
llm = LLM("gpt-3.5-turbo")


In [2]:
from agential.cog.critic.prompts import CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL, CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL, CRITIC_POT_INSTRUCTION_HUMANEVAL, HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC, HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL
from agential.cog.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT


inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
tests = f"{inst['test']}\ncheck({inst['entry_point']})"

use_tool = True

strat = CritHEvalCodeStrategy(llm=llm)
out = strat.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_HUMANEVAL,
    critique_examples=HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC if use_tool else HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL if use_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL,
    additional_keys={},
    critique_additional_keys={"tests": tests},
    max_interactions=3,
    use_tool=use_tool,
    reset=True
)

TypeError: Can't instantiate abstract class CritHEvalCodeStrategy with abstract method generate

In [ ]:
from agential.cog.critic.prompts import CRITIC_CRITIQUE_INSTRUCTION_MBPP, CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP, CRITIC_POT_INSTRUCTION_MBPP, MBPP_FEWSHOT_EXAMPLES_CRITIC, MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL
from agential.cog.critic.strategies.code import CritMBPPCodeStrategy
from agential.cog.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_POT


question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""
use_tool = True

strat = CritMBPPCodeStrategy(llm=llm)

out = strat.generate(
    question=question,
    examples=MBPP_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_MBPP,
    critique_examples=MBPP_FEWSHOT_EXAMPLES_CRITIC if use_tool else MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_MBPP if use_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP,
    additional_keys={"tests": tests},
    critique_additional_keys={"tests": tests},
    max_interactions=3,
    use_tool=use_tool,
    reset=True
)